In [61]:
### FPL Bot v2r0
#
# FPL Bot V2r0 is a form based bot
# Selects the team with the highest points per game over the previous x weeks, where x is an input parameter
# x will default to 3


import json
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import logging
import requests
import pandas as pd
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpStatus, PULP_CBC_CMD
from tabulate import tabulate
from tqdm.auto import tqdm

import smtplib
from email.mime.text import MIMEText

logging.basicConfig(level=logging.INFO)
tqdm.pandas()

def get_gameweek_history(player_id):
    """
    Get gameweeek history from FPL API for given player ID
    
    Inputs: 
    - player_id: integer FPL ID of given player

    Outputs:
    - df_history: pandas dataframe containing points history for given player ID
    """
    url = f'https://fantasy.premierleague.com/api/element-summary/{player_id}/'
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return pd.json_normalize(response.json()['history'])
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for player {player_id}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on failure

def get_all_gameweek_histories(player_ids):
    """Uses multithreading to speed up FPL API calls"""
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(tqdm(executor.map(get_gameweek_history, player_ids), total=len(player_ids)))
    return pd.concat(results, ignore_index=True)

In [62]:
def get_fpl_data():
    """
    Load player, team, position, and points history data from the FPL API.

    Outputs:
    - df_players: pandas dataframe containing player data
    - df_teams: pandas dataframe containing team data
    - df_positions: pandas dataframe containing position data
    - df_points: pandas dataframe containing points history data
    """
    logging.info("Fetching FPL data.")

    url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        response_json = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching FPL data : {e}")

    df_players = pd.json_normalize(response_json['elements'])
    df_teams = pd.json_normalize(response_json['teams'])
    df_positions = pd.json_normalize(response_json['element_types'])

    df_points = get_all_gameweek_histories(df_players['id'])
    return df_players, df_teams, df_positions, df_points

In [63]:
df_players, df_teams, df_positions, df_points = get_fpl_data()

INFO:root:Fetching FPL data.
100%|██████████| 755/755 [00:14<00:00, 50.80it/s]


In [64]:
pd.set_option('display.max_columns',None)
df_players.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,removed,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,region,team_join_date,has_temporary_code,opta_code,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,mng_win,mng_draw,mng_loss,mng_underdog_win,mng_underdog_draw,mng_clean_sheets,mng_goals_scored,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,3,0.0,0.0,0,Fábio,0.0,1,False,Has joined Portuguese side FC Porto on loan fo...,2024-08-29T11:06:25.241953Z,54,438098.jpg,0.0,False,Ferreira Vieira,0.0,False,None,u,1,3,0,439,0,2798,6,0.0,0.0,Fábio Vieira,NaN,None,False,p438098,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,743,322,743,323,740,321,743,322,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,134,80,730,317,744,323,662,285,0.00,0.00
1,True,True,0.0,0.0,205651,0,0,-3,3,2,4,0.0,0.0,0,Gabriel,3.2,2,False,Knee injury - Unknown return date,2025-01-12T22:00:07.802845Z,67,205651.jpg,2.5,False,Fernando de Jesus,1.8,False,None,i,1,3,42,1237337,125,1320685,53441,0.5,6.3,G.Jesus,30.0,2022-07-04,False,p205651,600,3,2,2,5,0,0,0,4,0,0,6,152,154.4,119.5,255.0,52.6,6,3.05,0.52,3.57,5.82,0,0,0,0,0,0,0,258,24,201,25,77,25,183,25,NaN,,NaN,,NaN,,0.46,0.0,0.08,0.54,0.87,0.75,37,19,88,12,198,24,142,21,0.90,0.30
2,True,True,100.0,100.0,226597,0,0,4,-4,2,2,2.9,2.9,1,Gabriel,2.4,3,False,,2024-11-27T10:30:06.325759Z,64,226597.jpg,4.2,False,dos Santos Magalhães,28.7,False,None,a,1,3,85,4092159,90410,2532148,112173,0.4,13.3,Gabriel,30.0,2020-09-01,False,p226597,1718,3,2,6,19,0,0,0,3,0,0,7,315,462.4,156.5,260.0,88.0,20,2.45,1.00,3.45,17.97,0,0,0,0,0,0,0,42,13,161,35,74,3,85,12,NaN,,NaN,,NaN,,0.13,0.0,0.05,0.18,0.94,1.00,50,3,143,41,44,4,9,3,1.05,0.31
3,True,True,100.0,100.0,219847,0,0,-2,2,1,4,3.5,3.5,7,Kai,3.0,4,False,,2025-01-02T08:00:08.350130Z,78,219847.jpg,4.3,False,Havertz,8.0,False,None,a,1,3,82,2487039,33742,3180819,19524,0.4,10.5,Havertz,80.0,2023-06-28,False,p219847,1660,8,2,6,19,0,0,0,4,0,0,11,302,404.4,240.9,630.0,127.6,19,8.37,1.38,9.75,17.45,0,0,0,0,0,0,0,65,8,104,8,9,5,27,6,NaN,,NaN,,3.0,,0.45,0.0,0.07,0.52,0.95,1.03,13,5,101,14,41,11,44,11,1.03,0.33
4,True,False,0.0,0.0,463748,0,0,0,0,0,1,0.0,0.0,0,Karl,0.0,5,False,Loaned to Real Valladolid,2024-08-14T08:31:46.556082Z,40,463748.jpg,0.0,False,Hein,0.0,False,None,u,1,3,0,0,0,4372,20,0.0,0.0,Hein,NaN,None,False,p463748,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,560,63,541,43,505,30,560,63,NaN,,NaN,,NaN,,0.00,0.0,0.00,0.00,0.00,0.00,648,58,472,54,567,63,584,69,0.00,0.00


In [65]:
df_teams.head()

,code,draw,form,id,loss,name,played,points,position,short_name,strength,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,2,ARS,4,None,False,0,1235,1335,1220,1370,1250,1300,1
1,7,0,None,2,0,Aston Villa,0,0,8,AVL,3,None,False,0,1110,1215,1080,1160,1140,1270,2
2,91,0,None,3,0,Bournemouth,0,0,7,BOU,4,None,False,0,1195,1215,1130,1170,1260,1260,127
3,94,0,None,4,0,Brentford,0,0,11,BRE,3,None,False,0,1100,1195,1080,1080,1120,1310,130
4,36,0,None,5,0,Brighton,0,0,9,BHA,3,None,False,0,1140,1150,1120,1140,1160,1160,131


In [66]:
df_positions.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_select,squad_max_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,None,None,1,1,True,[12],76
1,2,Defenders,DEF,Defender,DEF,5,None,None,3,5,False,[],250
2,3,Midfielders,MID,Midfielder,MID,5,None,None,2,5,False,[],328
3,4,Forwards,FWD,Forward,FWD,3,None,None,1,3,False,[],81
4,5,Managers,MNG,Manager,MNG,0,None,None,0,0,True,[],20


In [67]:
df_points[df_points['element'] == 4].head(20)

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,modified,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,mng_win,mng_draw,mng_loss,mng_underdog_win,mng_underdog_draw,mng_clean_sheets,mng_goals_scored,value,transfers_balance,selected,transfers_in,transfers_out
66,4,2,20,12,True,2024-08-17T14:00:00Z,2,0,1,False,90,1,1,1,0,0,0,0,0,0,0,3,48,54.8,24.1,46.0,12.5,1,0.45,0.04,0.49,0.47,0,0,0,0,0,0,0,80,0,1087445,0,0
67,4,11,2,2,False,2024-08-24T16:30:00Z,0,2,2,False,90,0,0,1,0,0,0,0,0,0,0,0,-2,0.0,6.1,15.0,2.0,1,0.21,0.03,0.24,1.28,0,0,0,0,0,0,0,81,285826,1638743,350954,65128
68,4,21,5,8,True,2024-08-31T11:30:00Z,1,1,3,False,90,1,0,0,1,0,0,0,0,0,0,2,34,47.0,24.8,30.0,10.2,1,0.77,0.05,0.82,1.83,0,0,0,0,0,0,0,81,63441,1755160,307787,244346
69,4,39,18,2,False,2024-09-15T13:00:00Z,0,1,4,False,90,0,0,1,0,0,0,0,0,0,0,0,9,19.2,5.4,36.0,6.1,1,0.14,0.00,0.14,0.74,0,0,0,0,0,0,0,81,-8676,1804099,301692,310368
70,4,47,13,2,False,2024-09-22T15:30:00Z,2,2,5,False,90,0,0,0,2,0,0,0,0,0,0,0,8,15.0,0.0,17.0,3.2,1,0.06,0.00,0.06,2.16,0,0,0,0,0,0,0,81,-82509,1733362,108350,190859
71,4,51,11,6,True,2024-09-28T14:00:00Z,4,2,6,False,90,1,0,0,2,0,0,0,0,0,0,0,25,35.6,8.5,100.0,14.4,1,1.89,0.04,1.93,0.32,0,0,0,0,0,0,0,82,201708,1961057,419703,217995
72,4,61,17,8,True,2024-10-05T14:00:00Z,3,1,7,False,90,1,0,0,1,0,0,0,0,0,0,2,34,37.6,14.9,82.0,13.5,1,0.77,0.04,0.81,0.65,0,0,0,0,0,0,0,82,177244,2162410,293506,116262
73,4,71,3,2,False,2024-10-19T16:30:00Z,2,0,8,False,90,0,0,0,2,0,0,0,0,0,0,0,4,3.6,3.3,10.0,1.7,1,0.04,0.02,0.06,1.82,0,0,0,0,0,0,0,83,-176824,1980343,105725,282549
74,4,81,12,2,True,2024-10-27T16:30:00Z,2,2,9,False,90,0,0,0,2,0,0,0,0,0,0,0,3,4.4,25.3,17.0,4.7,1,0.17,0.47,0.64,0.85,0,0,0,0,0,0,0,82,-214525,1768961,30424,244949
75,4,96,15,1,False,2024-11-02T12:30:00Z,1,0,10,False,90,0,0,0,1,0,0,0,1,0,0,0,4,5.6,11.8,4.0,2.1,1,0.00,0.42,0.42,0.53,0,0,0,0,0,0,0,81,-158755,1625586,45811,204566


In [68]:
# Data required to calculate total points in last x gameweeks:
#
# filter df_points for last x gameweeks
# from df_points:
#   total_points
#
# from df_positions:
#   singular_name_short
# 
# from df_teams:
#   g

In [108]:
def calc_points_per_game(df, num_gws=3):
    # filter for last x gameweeks
    df = df[df['round'] > df['round'].max() - num_gws]

    # group by player
    df_ppg = df.groupby(by='element', as_index=False)['total_points'].mean()
    df_ppg.rename(columns={'total_points':f'mean_ppg_last_{num_gws}'}, inplace=True)
    return df_ppg

In [109]:
df_ppg = calc_points_per_game(df_points,num_gws=7)
df_ppg.head()

,element,mean_ppg_last_7
0,1,0.000000
1,2,4.714286
2,3,3.571429
3,4,3.285714
4,5,0.000000


In [110]:
def process_fpl_data(df_players, 
                     df_positions, 
                     df_teams,
                     df_points,
                    ):
    # specify columns to persist for each dataframe
    # df_players
    players_cols_to_keep = [
        'id',
        'web_name',
        'chance_of_playing_next_round',
        'element_type',
        'now_cost',
        'team_code',
    ]

    # df_positions
    positions_cols_to_keep = [
        'id',
        'singular_name_short',
        'squad_select',
    ]

    # df_teams
    teams_cols_to_keep = [
        'code',
        'name',
    ]         

    num_gws = 7
    df_ppg_7 = calc_points_per_game(df_points,num_gws=num_gws)

    # merge reduced dataframes
    df = df_players[players_cols_to_keep].merge(
        df_positions[positions_cols_to_keep],
        how='inner',
        left_on='element_type',
        right_on='id',
        suffixes=['','_pos']
    ).merge(
        df_teams[teams_cols_to_keep],
        how='inner',
        left_on='team_code',
        right_on='code',
        suffixes=['','_team']
    ).merge(
        df_ppg_7,
        how='inner',
        left_on='id',
        right_on='element',
        suffixes=['_player','']
    )

    # rename columns for readability
    cols_to_rename = {
        'id':'id_player',
        'singular_name_short':'player_position',
        'name':'team_name',
        f'mean_ppg_{num_gws}':'form', 
    }

    df.rename(columns=cols_to_rename, inplace=True)
    return df

In [111]:
df = process_fpl_data(df_players, df_positions, df_teams, df_ppg)
df.head()

,id_player,web_name,chance_of_playing_next_round,element_type,now_cost,team_code,id_pos,player_position,squad_select,code,team_name,element,mean_ppg_last_7
0,1,Fábio Vieira,0.0,3,54,3,3,MID,5,3,Arsenal,1,0.000000
1,2,G.Jesus,0.0,4,67,3,4,FWD,3,3,Arsenal,2,4.714286
2,3,Gabriel,100.0,2,64,3,2,DEF,5,3,Arsenal,3,3.571429
3,4,Havertz,100.0,4,78,3,4,FWD,3,3,Arsenal,4,3.285714
4,5,Hein,0.0,1,40,3,1,GKP,2,3,Arsenal,5,0.000000


In [80]:
def select_fpl_squad(df,
                     metric,
                     num_gks=2,
                     num_defs=5,
                     num_mids=5,
                     num_atts=3,
                     max_value=1000):
    """
    Selects the optimal 15-man squad based on the given constraints.
    
    Inputs:
        - df: pandas dataframe containing player data
        - metric: column name to be optimised
        - num_gks: number of goalkeepers to be selected in squad
        - num_defs: number of defenders to be selected in squad
        - num_mids: number of midfielders to be selected in squad
        - num_atts: number of attackers to be selected in squad
        - max_value: maximum team value (multiplied by 10 so no decimal points)
    
    Returns:
        - selected_squad: pandas dataframe containing the selected squad
    """
    logging.info("Selecting optimal FPL squad.")

    # Reset index to ensure it ranges from 0 to N-1
    df = df.reset_index(drop=True)
    df.set_index('id_player',inplace=True)
    player_ids = df.index

    # Convert 'value' and 'P' columns to numeric
    df['now_cost'] = pd.to_numeric(df['now_cost'])
    df[metric] = pd.to_numeric(df[metric])

    # Define the LP problem
    prob = LpProblem("FPL_Squad_Selection", LpMaximize)

    # Define binary decision variables for each player
    x = LpVariable.dicts('x', player_ids, cat='Binary')

    # Objective function: maximize total points
    prob += lpSum(df.loc[i, metric] * x[i] for i in player_ids), "Total_Points"

    # Total value constraint
    prob += lpSum(df.loc[i, 'now_cost'] * x[i] for i in player_ids) <= max_value, "Total_value"

    # Position constraints
    positions = {'GKP': num_gks, 'DEF': num_defs, 'MID': num_mids, 'FWD': num_atts}
    for pos, count in positions.items():
        pos_ids = df[df['player_position'] == pos].index.tolist()
        prob += lpSum(x[i] for i in pos_ids) == count, f"Total_{pos.upper()}"

    # Team constraints: no more than 3 players from each team
    for team in df['team_name'].unique():
        team_ids = df[df['team_name'] == team].index.tolist()
        prob += lpSum(x[i] for i in team_ids) <= 3, f"Team_{team}"

    # Add constraints to ensure each player is selected at most once
    for i in player_ids:
        prob += x[i] <= 1, f"Select_{i}_At_Most_Once"

    # Solve the problem
    prob.solve(PULP_CBC_CMD(msg=0))

    # Check if an optimal solution was found
    if LpStatus[prob.status] != 'Optimal':
        logging.info("No optimal solution found.")
        return None
    else:
        logging.info("Optimal solution found.")


    # Get the selected players
    selected_ids = [i for i in player_ids if x[i].varValue == 1]
    selected_squad = df.loc[selected_ids].reset_index(drop=True)

    return selected_squad

In [81]:
squad = select_fpl_squad(df, 'form')

INFO:root:Selecting optimal FPL squad.
INFO:root:Optimal solution found.


In [84]:
def save_selected_squad(squad):
    """
    Print selected squad using tabulate and save to csv file

    Inputs:
        - squad: pandas dataframe containing 15 man squad
    """
    cols_to_print = [
        'web_name',
        'team_name',
        'mean_ppg',
        'player_position',
        'now_cost',
        'chance_of_playing_next_round',

    ]
    squad['player_position'] = pd.Categorical(squad['player_position'], ['GKP','DEF','MID','FWD'])
    squad = squad.sort_values(by=['player_position','mean_ppg']).round(2)
    print(tabulate(squad[cols_to_print], 
                   headers='keys',
                   tablefmt='psql'
                   ))

In [85]:
save_selected_squad(squad)

+----+------------+----------------+------------+-------------------+------------+--------------------------------+
|    | web_name   | team_name      |   mean_ppg | player_position   |   now_cost |   chance_of_playing_next_round |
|----+------------+----------------+------------+-------------------+------------+--------------------------------|
| 13 | Sels       | Nott'm Forest  |       5.14 | GKP               |         50 |                            100 |
|  9 | Dúbravka   | Newcastle      |       5.57 | GKP               |         43 |                            100 |
|  0 | Kerkez     | Bournemouth    |       5    | DEF               |         48 |                            100 |
|  2 | Huijsen    | Bournemouth    |       5    | DEF               |         44 |                            nan |
|  4 | Mitchell   | Crystal Palace |       5.29 | DEF               |         48 |                            nan |
| 12 | Aina       | Nott'm Forest  |       6.14 | DEF               |   

In [104]:
def email_squad(squad):
    positions = ['GKP', 'DEF', 'MID', 'FWD']
    final_str = ""
    buffer_str = "\n"

    for pos in positions:
        df_pos = squad[squad['player_position'] == pos]
        intro_str = f"The {pos}s selected are:\n"
        final_str += intro_str
        for _, row in df_pos.iterrows():
            name = row['web_name']
            team = row['team_name']
            cost = row['now_cost'] / 10
            mean_ppg = round(row['mean_ppg'], 2)

            player_str = f"| {name} | {team} |\n| Cost: {cost}m | Points per Game: {mean_ppg} |\n\n"
            final_str += player_str
        final_str += buffer_str

    subject = "Sven Botman's FPL Team of the Week"
    sender = "sven.fpl.botman@gmail.com"
    recipients = ["leon.wicks@btinternet.com"]
    password = "dtwl gdxe oifi zloj"


    msg = MIMEText(final_str)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
        smtp_server.login(sender, password)
        smtp_server.sendmail(sender, recipients, msg.as_string())
    
    logging.info("Email sent!")

email_squad(squad)

INFO:root:Fetching FPL data.
